In [314]:
import torch.nn as nn
import torch
import torch.backends.cudnn as cudnn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import scipy
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold as skf

In [315]:
import os
import torch.backends.cudnn as cudnn
import torch.utils.data


def test(dataset_loader,epoch,model,name):
    cuda = True
    cudnn.benchmark = True
    image_size = 32
    alpha = 0

    dataloader = dataset_loader

    """ training """

    my_net = model
    my_net = my_net.eval()

    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(dataloader)
    data_target_iter = iter(dataloader)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_dataloader:

        # test model using target data
        data_target = next(data_target_iter)
        test_img,test_wave, test_label = data_target

        batch_size = len(test_label)

        if cuda:
            test_img = test_img.cuda()
            test_wave = test_wave.cuda()
            test_label = test_label.cuda()

        class_output, _ = my_net(input_data=test_img.float(), input_wave = test_wave.float(), alpha=alpha)
        _,pred = torch.max(class_output, dim=1)
        _,gt = torch.max(test_label,dim=1)
        n_correct += (pred == gt).sum().item()
        n_total += batch_size

        i += 1

    accu = 100*n_correct/ n_total
    print(f'Epoch {epoch} for {name} dataset is {accu}')
    return accu


In [316]:
import torch.nn as nn
from torch.autograd import Function


class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(ResidualBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.activation1 = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride= 1, padding= 2)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.activation2 = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride= 1, padding= 2)
    
    def forward(self,x):
        residual  = torch.clone(x)
        x = self.bn1(x)
        x = self.activation1(x)
        x = self.conv1(x)
        x = self.conv2(self.activation2(self.bn2(x)))
        residual = residual.unsqueeze(0)
        residual = nn.functional.interpolate(residual, size = [x.shape[1], x.shape[2], x.shape[3]])
        residual = residual.squeeze(0)
        x += residual
        return x
    

class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 384
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 32), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))

    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)

        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)
        # FC Layer
        x = x.reshape(-1, 4*2*24)
        return x
    
    
class CNNModel(nn.Module):

    def __init__(self):
        super(CNNModel, self).__init__()
        self.feature_image = nn.Sequential()
        self.feature_image.add_module('f_conv1', nn.Conv2d(in_channels = 5, out_channels = 32, kernel_size= 3, stride= 1, padding= 2))
        self.feature_image.add_module('f_resblock1', ResidualBlock(32,32,5))
        self.feature_image.add_module('f_resblock2', ResidualBlock(32,64,5))
        self.feature_image.add_module('f_resblock3', ResidualBlock(64,128,5))
        self.feature_image.add_module('f_adaptiveavgpool', nn.AdaptiveAvgPool2d((15,15)))

        self.feature_wave = nn.Sequential()
        self.feature_wave.add_module('f_EEGNet',EEGNet())

        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1', nn.Linear(15 * 15 * 128 + 4*2*24, 1024))
        self.class_classifier.add_module('c_bn1', nn.BatchNorm1d(1024))
        self.class_classifier.add_module('c_relu1', nn.ReLU(True))
        self.class_classifier.add_module('c_drop1', nn.Dropout(0.2))
        self.class_classifier.add_module('c_fc2', nn.Linear(1024, 512))
        self.class_classifier.add_module('c_bn2', nn.BatchNorm1d(512))
        self.class_classifier.add_module('c_relu2', nn.ReLU(True))
        self.class_classifier.add_module('c_fc3', nn.Linear(512, 2))
        self.class_classifier.add_module('c_softmax', nn.Softmax(dim = 1))

        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(15 * 15 * 128 + 4*2*24, 1024))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(1024))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(1024, 2))
        self.domain_classifier.add_module('d_softmax', nn.Softmax(dim = 1))
    

    def forward(self, input_data, input_wave, alpha):
        feature1 = self.feature_image(input_data)
        feature2 = self.feature_wave(input_wave)
        feature1 = feature1.view(-1, 15 * 15 * 128)
        feature = torch.cat((feature1,feature2),1)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        print(feature.shape)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)
        return class_output, domain_output


In [326]:
model = CNNModel()
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")

Number of parameters: 60733836


In [343]:
class Dataset(Dataset):
    def __init__(self, data_img,data_wave,info):
        #data loading
        self.x1 = data_img
        self.x2 = data_wave
        self.y = info
        self.n_samples = data_img.shape[0]


    def __getitem__(self,index):
        t1 = self.x1[index]
        t3 = self.x2[index]
        t2 = self.y[index]
        t1 = torch.tensor(t1)
        t1 = t1.permute((2,0,1))
        t2 = torch.tensor(t2)
        t3 = torch.tensor(t3)
        return (t1,t3,t2)
    
    def __len__(self):
        return self.n_samples
    
data = scipy.io.loadmat('/home/desktop/Desktop/22104412_Docs/EEG-COGMusic/Baseline/datasets/s08_datasets_Zscore_clipped.mat')

x_c=data['coh']
x_p = data['pli']
x_d = data['psd']
labels_skf = data['labels_kfold']
labels = data['valence']
de = data['EEGNet']

In [344]:
model

CNNModel(
  (feature_image): Sequential(
    (f_conv1): Conv2d(5, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (f_resblock1): ResidualBlock(
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation1): ReLU()
      (conv1): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation2): ReLU()
      (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
    (f_resblock2): ResidualBlock(
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation1): ReLU()
      (conv1): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation2): ReLU()
      (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2

In [345]:
x_n = np.zeros((40,75,32,32,5))
for trial in range(x_n.shape[0]):
    for sample in range(x_n.shape[1]):
        x_n[trial,sample,:,:,0] = x_c[trial,sample,:,:,0]+np.transpose(x_p[trial,sample,:,:,0])
        x_n[trial,sample,:,:,1] = x_c[trial,sample,:,:,1]+np.transpose(x_p[trial,sample,:,:,1])
        x_n[trial,sample,:,:,2] = x_c[trial,sample,:,:,2]+np.transpose(x_p[trial,sample,:,:,2])
        x_n[trial,sample,:,:,3] = x_c[trial,sample,:,:,3]+np.transpose(x_p[trial,sample,:,:,3])
        x_n[trial,sample,:,:,4] = x_c[trial,sample,:,:,4]+np.transpose(x_p[trial,sample,:,:,4])
    print(f'Completed trial {trial}')

for trial in range(x_n.shape[0]):
    for sample in range(x_n.shape[1]):
        for i in range(32):
            x_n[trial,sample,i,i,:] = x_d[trial,sample,i,:]
    print(f'Completed trial {trial}')

Completed trial 0
Completed trial 1
Completed trial 2
Completed trial 3
Completed trial 4
Completed trial 5
Completed trial 6
Completed trial 7
Completed trial 8
Completed trial 9
Completed trial 10
Completed trial 11
Completed trial 12
Completed trial 13
Completed trial 14
Completed trial 15
Completed trial 16
Completed trial 17
Completed trial 18
Completed trial 19
Completed trial 20
Completed trial 21
Completed trial 22
Completed trial 23
Completed trial 24
Completed trial 25
Completed trial 26
Completed trial 27
Completed trial 28
Completed trial 29
Completed trial 30
Completed trial 31
Completed trial 32
Completed trial 33
Completed trial 34
Completed trial 35
Completed trial 36
Completed trial 37
Completed trial 38
Completed trial 39
Completed trial 0
Completed trial 1
Completed trial 2
Completed trial 3
Completed trial 4
Completed trial 5
Completed trial 6
Completed trial 7
Completed trial 8
Completed trial 9
Completed trial 10
Completed trial 11
Completed trial 12
Completed tri

In [346]:
v = labels_skf[:,0]
indices = np.where((v>5.5)|(v<4.5))[0]
indices.shape

(29,)

In [347]:
dt = x_n[indices]
labels = labels[indices]
trainLoss={}
valLoss = {}
trainAcc = {}
valAcc = {}

In [348]:
l_skf = np.zeros([40,1])
l_skf[np.where(labels_skf[:,0]>5)[0]] = 1
l_skf = l_skf[indices]

In [349]:
dt.shape,labels.shape,l_skf.shape, 

((29, 75, 32, 32, 5), (29, 75, 2), (29, 1))

In [350]:
de = de.transpose((0,1,3,2))
de = de[indices,:,np.newaxis,:,:]
de.shape

(29, 75, 1, 384, 32)

In [351]:
kf = skf(n_splits = 10)
log_pred_dict = {}
for k,(train_index,test_index) in enumerate(kf.split(dt, l_skf)):
    print(f'Fold {k+1} running')
    deTr,deV = np.concatenate(de[train_index],0), np.concatenate(de[test_index],0)
    dataTr, dataV = np.concatenate(dt[train_index],0), np.concatenate(dt[test_index],0)
    labelsTr, labelsV = np.concatenate(labels[train_index],0), np.concatenate(labels[test_index],0)
    tAV = []
    vAV = []
    y_train = np.argmax(labelsTr, axis=1)
    class_counts = np.bincount(y_train)
    num_classes = len(class_counts)
    total_samples = len(y_train)

    class_weights = []
    for count in class_counts:
        weight = 1 / (count / total_samples)
        class_weights.append(weight)
    
    weights_class = torch.tensor(np.array(class_weights))
    weights_class = weights_class.cuda()

    weights_domain = torch.tensor(np.array([1.05,5.5]))
    ## parameters
    bs = 100
    image_size = 32
    n_epoch = 50

    ## load data
    dataset_source = Dataset(dataTr, deTr, labelsTr)
    dataloader_source = DataLoader(dataset = dataset_source, batch_size = bs, shuffle = True)
 
    dataset_target = Dataset(dataV,deV, labelsV)
    dataloader_target = DataLoader(dataset = dataset_target, batch_size = 10, shuffle = True)
    
    testDS = Dataset(dataV,deV,labelsV)
    testDL = DataLoader(dataset = testDS, batch_size = bs)

    ###### model
    my_net = CNNModel()
     # Define your weights as a tensor
    optimizer = optim.SGD(my_net.parameters(), lr=0.0001, momentum= 0.9)
    loss_class = nn.CrossEntropyLoss(weight=weights_class)
    loss_domain = nn.CrossEntropyLoss(weight=weights_domain)

    cuda = True
    cudnn.benchmark = True

    if cuda:
        my_net = my_net.cuda()
        loss_class = loss_class.cuda()
        loss_domain = loss_domain.cuda()

    for p in my_net.parameters():
        p.requires_grad = True

    #### training    
    for epoch in range(n_epoch):

        len_dataloader = min(len(dataloader_source), len(dataloader_target))
        data_source_iter = iter(dataloader_source)
        data_target_iter = iter(dataloader_target)

        i = 0
        while i < len_dataloader:

            p = float(i + epoch * len_dataloader) / n_epoch / len_dataloader
            alpha = 2. / (1. + np.exp(-10 * p)) - 1

            # training model using source data
            data_source = next(data_source_iter)
            source_img, source_wave, source_label = data_source

            my_net.zero_grad()
            batch_size = len(source_label)

            domain_label = torch.zeros(batch_size,2)
            domain_label[:,0] = 1
            domain_label = domain_label.long()

            if cuda:
                source_img = source_img.cuda()
                source_wave = source_wave.cuda()
                source_label = source_label.cuda()
                domain_label = domain_label.cuda()

            class_output, domain_output = my_net(input_data=source_img.float(), input_wave = source_wave.float(), alpha=alpha)
            _, class_label = torch.max(source_label,dim=1)
            err_s_label = loss_class(class_output, source_label.float())
            err_s_domain = loss_domain(domain_output, domain_label.float())

            # training model using target data
            data_target = next(data_target_iter)
            target_img,target_wave, _ = data_target

            batch_size = len(target_img)

            domain_label = torch.zeros(batch_size,2)
            domain_label[:,1] = 1
            domain_label = domain_label.long()

            if cuda:
                target_img = target_img.cuda()
                target_wave = target_wave.cuda()
                domain_label = domain_label.cuda()

            _, domain_output = my_net(input_data=target_img.float(), input_wave = target_wave.float(), alpha=alpha)
            err_t_domain = loss_domain(domain_output, domain_label.float())
            err = err_t_domain + err_s_domain + err_s_label
            err.backward()
            optimizer.step()

            i += 1

            print ('epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' \
                % (epoch, i, len_dataloader, err_s_label.cpu().data.numpy(),
                    err_s_domain.cpu().data.numpy(), err_t_domain.cpu().data.numpy()))

        tAV.append(test(dataloader_source, epoch, my_net, 'source'))
        vAV.append(test(dataloader_target, epoch, my_net, 'target'))
    trainAcc[str(k)] = tAV
    valAcc[str(k)] = vAV
    print(f'Fold {k} Completed')

Fold 1 running
torch.Size([100, 28992])
torch.Size([10, 28992])
epoch: 0, [iter: 1 / all 20], err_s_label: 1.366622, err_s_domain: 0.735890, err_t_domain: 3.941936
torch.Size([100, 28992])
torch.Size([10, 28992])
epoch: 0, [iter: 2 / all 20], err_s_label: 1.404003, err_s_domain: 0.739976, err_t_domain: 3.790384
torch.Size([100, 28992])
torch.Size([10, 28992])
epoch: 0, [iter: 3 / all 20], err_s_label: 1.389585, err_s_domain: 0.759002, err_t_domain: 3.684385
torch.Size([100, 28992])
torch.Size([10, 28992])
epoch: 0, [iter: 4 / all 20], err_s_label: 1.402417, err_s_domain: 0.786101, err_t_domain: 3.556102
torch.Size([100, 28992])
torch.Size([10, 28992])
epoch: 0, [iter: 5 / all 20], err_s_label: 1.376137, err_s_domain: 0.818004, err_t_domain: 3.378201
torch.Size([100, 28992])
torch.Size([10, 28992])
epoch: 0, [iter: 6 / all 20], err_s_label: 1.378894, err_s_domain: 0.852640, err_t_domain: 3.289988
torch.Size([100, 28992])
torch.Size([10, 28992])
epoch: 0, [iter: 7 / all 20], err_s_label:

In [352]:
target_img.float()

tensor([[[[2.2779e+00, 5.9574e-01, 3.5781e-01,  ..., 3.5448e-01,
           4.3071e-01, 6.3914e-01],
          [7.8125e-02, 5.7037e+00, 8.8726e-01,  ..., 1.9957e-01,
           9.4475e-01, 9.5497e-01],
          [3.1250e-02, 3.6458e-02, 4.6568e+00,  ..., 1.1359e-01,
           9.6055e-01, 8.1226e-01],
          ...,
          [1.5625e-02, 9.3750e-02, 1.0417e-01,  ..., 5.2649e+00,
           1.4409e-01, 1.8527e-01],
          [6.7708e-02, 3.1250e-02, 2.0833e-02,  ..., 3.1250e-02,
           5.4653e+00, 8.8539e-01],
          [1.5625e-01, 5.2083e-03, 1.5625e-02,  ..., 1.5625e-02,
           3.1250e-02, 6.3023e+00]],

         [[1.9474e+00, 7.5450e-01, 6.8079e-01,  ..., 2.0397e-01,
           6.8857e-01, 6.4750e-01],
          [9.3750e-02, 2.1922e+00, 8.8815e-01,  ..., 1.0430e-01,
           9.1100e-01, 8.9847e-01],
          [6.6667e-01, 7.1875e-01, 2.3238e+00,  ..., 1.4875e-01,
           8.6037e-01, 7.3858e-01],
          ...,
          [2.8646e-01, 3.9583e-01, 2.5521e-01,  ..., 1.1310

In [353]:
class_output.shape

torch.Size([100, 2])

In [354]:
vAV = np.array(list(valAcc.values()))

In [355]:
VA = np.max(vAV,1)
print(VA)

[74.22222222 70.66666667 81.33333333 85.33333333 63.55555556 60.88888889
 52.         60.88888889 46.22222222 56.        ]


In [356]:
np.mean(VA)

65.11111111111111